# LSTM

Test of simple MLP models with different amount of neurons, hidden layers and size of input vector

## Load Data

In [72]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn

import pandas as pd
import numpy as np

from tqdm import tqdm

import plotly.express as px
from torch.optim import Adam


import matplotlib.pyplot as plt

In [73]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [74]:
df = pd.read_csv("data_N_6000_noise_level_0.05.csv")
df["Time"] = 1
df_diff = df
#df_diff = pd.DataFrame(df.iloc[1:].values - df.iloc[:-1].values, columns=["dt", "dx", "dy", "dz"])

In [75]:
class SequentDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, n_dots=1, n_dot_parameters=4):
        self.n_dot_parameters = n_dot_parameters
        self.n_dots = n_dots
        self.X_, self.y_ = self.__make_stack(dataframe)
        print(self.X_.shape)
        print(self.y_.shape)

    def __len__(self):
        return len(self.X_)

    def __getitem__(self, idx):
        return self.X_[idx], self.y_[idx]

    def __make_stack(self, df: pd.DataFrame):
        seq_amount = df.shape[0] -  self.n_dots - 1
        X = torch.zeros((seq_amount, self.n_dots, self.n_dot_parameters), dtype=torch.float32)
        y = torch.zeros((seq_amount, self.n_dots, self.n_dot_parameters), dtype=torch.float32)
        for i in range(seq_amount):
            X[i, :, :] = torch.reshape(torch.tensor(df.values[i:i+self.n_dots, :], dtype=torch.float32), (1, self.n_dots, self.n_dot_parameters))
            y[i, :, :] = torch.reshape(torch.tensor(df.values[i+1:i+self.n_dots+1, :], dtype=torch.float32), (1, self.n_dots, self.n_dot_parameters))

        # stacks = [[df.iloc[:-self.n_dots]]] + [df.iloc[i:].values if (self.n_dots == i) else df.iloc[i:-(self.n_dots - i)].values for i in range(1, self.n_dots + 1)]
        return (X, y)

In [76]:
N_DOTS = 100
BATCH_SIZE = 8
N_DOT_PARAMETERS = 4
N_LSTM_LAYERS = 1

In [77]:
dataset = SequentDataset(df_diff[0:5000], n_dots=N_DOTS, n_dot_parameters=N_DOT_PARAMETERS)
test = SequentDataset(df_diff[5000:6000], n_dots=N_DOTS, n_dot_parameters=N_DOT_PARAMETERS)

torch.Size([4899, 100, 4])
torch.Size([4899, 100, 4])
torch.Size([899, 100, 4])
torch.Size([899, 100, 4])


In [78]:
train_data, val_data = random_split(dataset,[0.8,0.2])

train_loader = DataLoader(train_data, batch_size=N_DOTS, shuffle=True)
val_loader = DataLoader(val_data, batch_size=N_DOTS, shuffle=False)
test_loader = DataLoader(test, batch_size=N_DOTS, shuffle=False)

## Create Model

In [79]:
class RNNModel(nn.Module):
    def __init__(self, input_size=4, hidden_layer_size=100, output_size=4, num_layers=10):
        super(RNNModel, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.rnn = nn.RNN(input_size, hidden_layer_size, num_layers, batch_first=True)

        # Define the output layer
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, x):
        x = x.to(device)
        batch_size = x.size(0)
        self.h = torch.zeros(self.num_layers, batch_size, self.hidden_layer_size).requires_grad_()
        self.h = self.h.to(device)
        out, _ = self.rnn(x, self.h)

        # print(out.size())

        # Pass through fully connected layer
        out = self.linear(out)  # We want the output corresponding to the last time step
        return out


In [80]:
model = RNNModel(input_size=N_DOT_PARAMETERS, num_layers=N_LSTM_LAYERS).to(device)
loss_model = nn.MSELoss(reduction='mean')
# loss_model = nn.L1Loss(reduction='mean')
opt = Adam(model.parameters(), lr=0.001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=5, gamma=0.95)

In [81]:
model

RNNModel(
  (rnn): RNN(4, 100, batch_first=True)
  (linear): Linear(in_features=100, out_features=4, bias=True)
)

In [82]:
EPOCH = 250
train_losses = []
val_losses = []

for epoch in range(EPOCH):

    # Обучение
    model.train()
    train_loss = []
    for X, y in train_loader:
        X = X.to(device)
        y = y.to(device)

        y_pred = model(X)
        loss = loss_model(y_pred, y)
        train_loss.append(loss.item())

        opt.zero_grad()
        loss.backward()

        opt.step()
        mean_train_loss = sum(train_loss)/len(train_loss)
        # train_loop.set_description(f"Epoch [{epoch+1}/{EPOCH}], train_loss = {mean_train_loss:.4f}")

    train_losses.append(mean_train_loss)

    # Валидация
    model.eval()
    with torch.no_grad():
        val_loss = []
        for X, y in val_loader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            loss = loss_model(pred, y)
            val_loss.append(loss.item())

        mean_val_loss = sum(val_loss)/len(val_loss)
        val_losses.append(mean_val_loss)

    lr_scheduler.step()
    lr = lr_scheduler.get_last_lr()
    print(f"Epoch [{epoch+1}/{EPOCH}], train_loss = {mean_train_loss:.4f}, val_loss = {mean_val_loss:.4f}")

Epoch [1/250], train_loss = 1.0448, val_loss = 0.1462
Epoch [2/250], train_loss = 0.0881, val_loss = 0.0467
Epoch [3/250], train_loss = 0.0321, val_loss = 0.0251
Epoch [4/250], train_loss = 0.0225, val_loss = 0.0208
Epoch [5/250], train_loss = 0.0195, val_loss = 0.0185
Epoch [6/250], train_loss = 0.0174, val_loss = 0.0167
Epoch [7/250], train_loss = 0.0157, val_loss = 0.0151
Epoch [8/250], train_loss = 0.0143, val_loss = 0.0137
Epoch [9/250], train_loss = 0.0130, val_loss = 0.0125
Epoch [10/250], train_loss = 0.0119, val_loss = 0.0114
Epoch [11/250], train_loss = 0.0109, val_loss = 0.0105
Epoch [12/250], train_loss = 0.0100, val_loss = 0.0097
Epoch [13/250], train_loss = 0.0092, val_loss = 0.0089
Epoch [14/250], train_loss = 0.0085, val_loss = 0.0082
Epoch [15/250], train_loss = 0.0078, val_loss = 0.0076
Epoch [16/250], train_loss = 0.0072, val_loss = 0.0070
Epoch [17/250], train_loss = 0.0067, val_loss = 0.0065
Epoch [18/250], train_loss = 0.0062, val_loss = 0.0060
Epoch [19/250], tra

In [83]:
predictions = []
model.eval()

for X, y in test_loader:
    with torch.no_grad():
        for i in range(N_DOTS, 1000):
            y_pred = model(X)
            predictions.append(y_pred[0, -1, :])
            X[0, :-1, :] = X.clone()[0, 1:, :]
            X[0, -1, :] = y_pred[0, -1, :]

    break

In [84]:
p_x = pd.DataFrame(np.vstack(list(map(lambda x: x.cpu().detach().numpy(), predictions))), columns=["Time", "X", "Y", "Z"])
_, y  = test[:1000]
p_y = pd.DataFrame(y.cpu().detach().numpy()[:, 0, :], columns=["Time", "X", "Y", "Z"])

In [85]:
p_x["C"] = "Predict"
p_y["C"] = "True"

d = pd.concat([p_x,p_y],axis=0)

In [86]:
fig = px.line_3d(d, x="X", y="Y", z="Z", color='C')
# fig = px.scatter_3d(data, x="X", y="Y", z="Z")
fig.show()